In [1]:
# Grupo 12
# Francisco de Borja Lozano del Moral
# Manuel Ortega Salvador

# Práctica 1.A Toma de contacto con AIMA

La práctica está organizada en 3 partes. En la primera se muestra a través de ejemplos cómo se **representan** algunos problemas clásicos como el de las jarras o el problema del ocho puzzle. En la segunda parte se muestra el uso de los algoritmos de búsqueda. En la tercera parte aprenderemos a medir las propiedades de los algoritmos.
En el notebook encontraras claramente identificados los lugares en los que debes incluir código o comentarios.  

Cuando termines los ejercicios entrega este archivo en el campus. 

## Parte I: Representación de problemas de espacios de estados.

### El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. En esta parte en vez de importarla la hemos copiado aquí para la explicación.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en:
* Representar estados y acciones mediante una estructura de datos.
* Definir: estado_inicial, es_estado_final(_), acciones(_), aplica(_,_) y
  coste_de_aplicar_accion, si el problema tiene coste.

 La siguiente clase Problem representa este esquema general de cualquier
 problema de espacio de estados. Un problema concreto será una subclase de
 Problema, y requerirá implementar acciones, aplica y eventualmente __init__, actions,
 goal_test. La función coste_de_aplicar_accion la hemos incluido nosotros.

In [6]:
   class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver un ejemplo de cómo definir un problema como subclase
de problema. En concreto, el problema de las jarras, visto en clase que es muy sencillo. 

In [7]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        accs=list()
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs

    def result(self,estado,accion):
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


Vamos a probar algunos ejemplos.

In [8]:
p =Jarras()
p.initial

(0, 0)

In [9]:
p.actions(p.initial)

['llenar jarra de 4', 'llenar jarra de 3']

In [10]:
p.result(p.initial,"llenar jarra de 4")

(4, 0)

In [11]:
p.coste_de_aplicar_accion(p.initial,"llenar jarra de 4")

1

In [12]:
p.goal_test(p.initial)

False

### Problema de los misioneros

In [13]:
# Creamos la clase ProblemaMisioneros con los elementos que representarán el problema. 
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        # cada accion tiene un texto para identificar al operador y despues una tupla de dos elementos con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)



In [14]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

# Asegurate de que entiendes la formalización del problema y haz algunas pruebas con la representación del problema de los misioneros. 
# En la siguiente parte vamos a usar las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura la llamada sería:  breadth_first_tree_search(estado).solution()

In [15]:
misioneros.initial

(3, 3, 0)

In [16]:
misioneros.actions(misioneros.initial)

[('1c', (0, 1)), ('2c', (0, 2)), ('1m1c', (1, 1))]

### Representación del problema del puzzle de 8

Vamos a definir la clase Ocho_Puzzle, que implementa la representación del problema del 8-puzzle visto en clase. 
Se os proporciona una versión incompleta y tendréis que completar el código que se presenta a continuación, en los lugares marcados con interrogantes.

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. **Tenlo en cuenta al hacer las pruebas**. 

### EJERCICIO 1. Completa la definición de los operadores en el problema del Puzle de 8. 

In [17]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        
        ### EJERCICIO 1.1. COMPLETA LA DEFINICIÓN DE LOS OPERADORES.
        if pos_hueco not in (0,3,6):
            accs.append("Mover hueco izquierda")
            
        if pos_hueco not in (2,5,8):
            accs.append("Mover hueco derecha")
            
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
         
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        
        ### EJERCICIO 1.2. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        elif accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
            
        elif accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
            
        elif accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+3]
            l[pos_hueco+3] = 0
        
        return tuple(l)
    
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1 

#### Una vez completada la definición de la clase podrás probar los siguientes ejemplos.

In [18]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [19]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [20]:
p8.result(p8.initial,"Mover hueco arriba")

(2, 8, 3, 1, 0, 4, 7, 6, 5)

In [21]:
p8.result(p8.initial,"Mover hueco abajo")
#Da error porque el hueco está en la última fila del tablero, luego mover abajo no es un movimiento válido

IndexError: list index out of range

In [22]:
p8.result(p8.initial,"Mover hueco derecha")

(2, 8, 3, 1, 6, 4, 7, 5, 0)

In [23]:
p8.coste_de_aplicar_accion(p8.initial,"Mover hueco abajo")

1

### Usaremos búsqueda en anchura y en profundidad para encontrar soluciones tanto al problema de las jarras, los misioneros y el problema del ocho puzzle con distintos estados iniciales.

In [24]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [25]:
## resolvemos el problema de las jarras con el método de búsqueda en anchura.  

In [26]:
breadth_first_tree_search(Jarras()).solution()

['llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3',
 'vaciar jarra de 3',
 'trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3']

In [27]:
depth_first_graph_search(Jarras()).solution()

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

### Ejercicio 2. Prueba los algoritmos de búsqueda ciega con el problema de los misioneros y con el  puzzle de 8

In [28]:
# Usaremos las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura breadth_first_tree_search(misioneros).solution()

In [29]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [30]:
p8.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [31]:
breadth_first_tree_search(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
# La llamada corresponde al algoritmo de busqueda en anchura sin control de repetidos.  
# Busqueda en anchura es completo.. ¿no debería terminar? escribe al final del ejercicio tus conclusiones.

KeyboardInterrupt: 

In [32]:
breadth_first_tree_search(Ocho_Puzzle((1,2,3,4,5,6,0,7,8))).solution()

['Mover hueco derecha', 'Mover hueco derecha']

In [33]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [34]:
breadth_first_tree_search(estado).solution()
# Respuesta: ['UP', 'LEFT', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'RIGHT', 'DOWN']

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [ ]:
%%timeit
breadth_first_tree_search(estado).solution()

In [ ]:
depth_first_graph_search(estado).solution()

In [ ]:
breadth_first_graph_search(estado).solution()

In [ ]:
%%timeit
breadth_first_graph_search(estado).solution()

#### En este ejercicio se ha podido observar los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega.  Escribe aquí tus conclusiones:

Búsqueda en anchura: 

En el primer caso, en el que la llamada en anchura no termina, lo que sucede es que esa configuración de tablero y objetivo no tiene solución y no hay detección de ciclos (la búsqueda en anchura siempre encuentra la solución, si la hay). 

breadth_first_tree_search no tiene por qué terminar porque no tiene manera de detectar y evitar ciclos. En este caso, al no haber solución explora todo el arbol de soluciones, pero al haber ciclos (no es un árbol), no puede terminar nunca de explorar el arbol. Cambiando a breadth_first_graph_search si terminaría, tras haber explorado todos los estados posibles.

En el caso del tablero (2, 4, 3, 1, 5, 6, 7, 8, 0), encuentra la solución óptima en 8 niveles. Tanto TreeSearch (sin control de repetidos) como GraphSearch (con control de repetidos) tardan muy poco a simple vista en encontrar la solución óptima. Si usamos la instrucción timeit, vemos que TreeSearch tarda 15.1 ms per loop, mientras que GraphSearch tarda solo 2.26 ms per loop, por lo que vemos que controlar repetidos reduce el tiempo de ejecución.
 
**********************************************************************************
Búsqueda en profundidad:

El algoritmo ha tardado aproximadamente 10 minutos en encontrar la solución. Seguramente se debe a que el algoritmo ha empezado por ramas cuya solución está a una profundidad muy elevada. Como estamos usando GraphSearch (control de repetidos) no se ha debido a que el algoritmo se haya encontrado con un ciclo.

### Ejercicio 3:  Definición de heurísticas

#### Para el problema de los misioneros define una heurística y comenta sus propiedades
Como vimos en la teoría, una heurística puede ser relajando el probema quitando la condición en la que los caníbales se comen a los misioneros. De este modo nos queda que en cada viaje llevamos a una persona (la otra tiene que llevar de vuelta la barca a la orilla inicial). Así, nos queda h(n) = 2*(Mi+Ci)-orilla(n), siendo Mi y Ci el número de misioneros y caníbales que quedan en la orilla inicial, respectivamente, y orilla(n)=1 si la barca está en la orilla inicial y orilla(n)=0 si está en la final.
Puede haber alguna otra heurística más informada, que no implique relajar el problema eliminando alguna condición, pero igualmente esta es una heurística perfectamente válida.
#### Para el problema del puzle de 8 se pide definir al menos las siguientes funciones heurísticas:
* linear(node): cuenta el número de casillas mal colocadas respecto al estado final.
* manhattan(node): suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.
* max_heuristic(node: maximo de las dos anteriores
* sqrt_manhattan(node):  raíz cuadrada de la distancia Manhattan

In [22]:
import math
goal = (1,2,3,4,5,6,7,8,0)

In [23]:
# Heuristicas para el 8 Puzzle. Puedes definir las funciones fuera de la clase ya que en la llamada a A* puedes pasar el nombre 
# de la función. 


def linear(node):
    #goal = node.state.goal
    state = node.state
    num = 0
    for i in range(9): 
        if state[i] != goal[i]:
            num += 1 
    return num

correct_position = list(range(9))
for i in range(9):
    correct_position[goal[i]] = i 
    
def manhattan(node):
    state = node.state
    mhd = 0;
    for i in range(3):
        for j in range(3):
            current = correct_position[state[j*3+i]]
            mhd += abs(i - (current) % 3) + abs(j - current // 3)
    return mhd

def sqrt_manhattan(node):
    mhd = manhattan(node)
    return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)
        

### Ejercicio 4. 
Usar las implementaciones de los algoritmos que correspondan a búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema del 8 puzzle para el siguiente **estado inicial** y comparar los costes temporales usando %timeit y comentar los resultados.

              +---+---+---+
              | 2 | 4 | 3 |
              +---+---+---+
              | 1 | 5 | 6 |
              +---+---+---+
              | 7 | 8 | H |
              +---+---+---+


In [24]:
puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0)) 
astar_search(puzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [25]:
astar_search(puzle,manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [26]:
puzle.initial

(2, 4, 3, 1, 5, 6, 7, 8, 0)

In [27]:
puzle.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [28]:
astar_search(puzle,linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [29]:
astar_search(puzle,max_heuristic).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

¿Has notado diferencias en los tiempos de ejecución? Vamos a medirlo.Aunque las heurísticas no afectan a la solución obtenida sí hay diferencias importantes en el tiempo de cálculo
<br>

In [30]:
# goal = (1, 2, 3, 4, 0, 5, 6, 7, 8)
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0), goal=goal)
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8), goal=goal)
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0), goal=goal)
# puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 0, 7, 8, 6, 0), goal=goal)

In [31]:
%%timeit
astar_search(puzzle_1, linear)
astar_search(puzzle_2, linear)
astar_search(puzzle_3, linear)

7.51 ms ± 707 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [32]:
%%timeit
astar_search(puzzle_1, manhattan)
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

3.78 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
%%timeit
astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

87.3 ms ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
%%timeit
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

7.16 ms ± 1.42 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Escribe aquí tus conclusiones sobre qué heurística es mejor y por qué.

# ----------------------------------------------------------------------------
Normalmente, cuanto más próximo al coste real, mejor es la heurística. Esto se puede apreciar en los resultados, ya que es cierto que manhattan es una métrica más próxima al coste real que lineal, ya que la métrica lineal no da información sobre qué estado está más cerca de la solución entre dos estados con la misma cantidad de casillas mal colocadas. 

Por otro lado, sqrt_manhattan, métrica próxima a la euclídea, solo sería mejor que manhattan si se pudieran hacer movimientos diagonales. Como en el puzzle el coste de una pieza es coste de manhattan ya que solo se pueden mover fichas en horizontal y vertical. sqrt_manhattan suaviza el error  y hace parecer que distancias diagonales grandes suponen un error menor del que realmente supone en el problema.

Respecto a porqué max_heuristic funciona de media, un poco peor, es porque manhattan siempre es mayor a lineal, luego hace las evaluaciones de linear innecesarias, ya que si una ficha está mal, como poco tiene distancia de manhattan 1, que exactamente la distancia lineal.

# ----------------------------------------------------------------------------


## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle. 
### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.
Usaremos la función %timeit para medir los tiempos y para el espacio una version modificada de Problema que almacena el número de nodos.



En la clase modificada también vamos a incluir el calculo que nos diga si un cierto tablero tiene solución o no. Esto es muy útil.. como hemos comentado al principio solo algunos tableros tienen solucion. En el caso del puzle de 8 un tablero se ha demostrado que se puede comprobar calculando su paridad (o número de inversiones). Si es impar el tablero **no** tiene solución.  El concepto de inversión de una ficha será la suma del número de fichas que se encuentran en una posición superior a dicha ficha y que deberían estar situadas en una posición inferior. La inversión total será la suma de las inversiones individuales. Si este número es par, el puzzle tendrá solución. En caso contrario, no habrá solución. 

In [3]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas, para resolver el 8-puzzle. 
# The solvability of a configuration can be checked by calculating the Inversion Permutation. If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)

NameError: name 'Problem' is not defined

In [4]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
p8 = Ocho_Puzzle(estado_inicial)

NameError: name 'Problema_con_Analizados' is not defined

In [5]:
p8p.initial

NameError: name 'p8p' is not defined

In [38]:
p8p.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [39]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
h2_ocho_puzzle = linear
astar_search(puzzle_1, h2_ocho_puzzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [40]:
astar_search(p8, h2_ocho_puzzle).solution()

['Mover hueco derecha']

In [41]:
astar_search(p8p, h2_ocho_puzzle).solution()

['Mover hueco derecha']

In [42]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """

    p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
    if p8p.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [43]:
resuelve_ocho_puzzle(estado_inicial,astar_search,h2_ocho_puzzle)

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 1. Nodos analizados: 2


### Ejercicio 5:  resolver usando las distintas búsquedas y en su caso, las distintas heurísticas, el problema del 8 puzzle para los siguientes estados iniciales:

E1 = (2,1,3,4,8,6,7,0,5)
E2 = (1,0,3,4,8,6,7,2,5)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)

          E1              E2              E3              E4
           
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    
     | 2 | 1 | 3 |   | 1 | 0 | 3 |   | 4 | 5 | 6 |   | 1 | 2 | 3 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
     | 4 | 8 | 6 |   | 4 | 8 | 6 |   | 1 | 0 | 3 |   | H | 5 | 6 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
     | 7 | H | 5 |   | 7 | 2 | 5 |   | 7 | 8 | 2 |   | 4 | 7 | 8 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
   
 Se pide, en cada caso, obtener detalles del tiempo y espacio necesario para la resolución de estos estados.
 Hacerlo con la función resuelve_ocho_puzzle, para
 obtener, además de la solución, la longitud (el coste) de la solución
 obtenida y el número de nodos analizados. Anotar los resultados en la
 siguiente tabla (L, longitud de la solución, NA, nodos analizados, T, tiempo, y
 **justificar los resultados con las distintas propiedades teóricas estudiadas en clase**.  
 
 

-----------------------------------------------------------------------------------------
                                       E1           E2           E3          E4
    Anchura                            L=17         L=11         L=20        L=3
                                       T=6.781s     T=0.0499s    T=91.097s   T=0.5ms  
                                       NA=14017     NA=990       NA=52804    NA=19 
   
    Profundidad                        L=64073      L=47931      L=3556      L=3  
                                       T=1009.12s   T=533.235s   T=2.2619s   T=0s 
                                       NA=76897     NA=54791     NA=3975     NA=4 
                                       
    Coste Uniforme                     L=17         L=11         L=20        L=3
                                       T=58.987s    T=0.365s     T=816.919s  T=0.99ms  
                                       NA=14092     NA=870       NA=48428    NA=13 
                                       
    Primero el mejor (linear)          L=17         L=11         L=20        L=3
                                       T=17.385s    T=0.152s     T=105.37s   T=0.3ms  
                                       NA=739697    NA=4359      NA=6500219  NA=4
                                                                            
    Primero el mejor (manhattan)       L=17         L=11         L=22        L=3 
                                       T=23.418s    T=0.0328s    T=54.433s   T=0.997ms  
                                       NA=573520    NA=1261      NA=2303999  NA=4
                                                                             
    A* (linear)                        L=17         L=11         L=20        L=3 
                                       T=0.231s     T=4ms        T=3.128s    T=0.997ms  
                                       NA=873       NA=77        NA=3377     NA=4
                                                                             
    A* (manhattan)                     L=17        L=11         L=20          L=3 
                                       T=0.042s    T=1ms        T=0.384s      T=0.997ms  
                                       NA=362      NA=39        NA=1153       NA=4
    
    h3                                 L=17        L=11         L=20          L=3 
                                       T=0.024s    T=1ms        T=0.011s      T=0.997ms  
                                       NA=194      NA=31        NA=129        NA=4
                                       
    h4                                 L=17        L=11         L=20          L=3 
                                       T=0.049s    T=0.9ms      T=5.9ms       T=0.s  
                                       NA=355      NA=29        NA=76         NA=4
                                       
    h5                                 L=17        L=11         L=20          L=3 
                                       T=0.097s    T=0.9ms      T=2.9ms       T=0s  
                                       NA=466      NA=22        NA=50         NA=4                                   
 -----------------------------------------------------------------------------------------

Mirando primero el caso E4, que es el más sencillo, vemos que todos obtienen la solución en un nivel de profundidad muy pequeño,  aunque coste uniforme y en profundidad tienen que analizar más nodos.

Fijándonos ahora en el caso E2, vemos que la mejor es A* con la heurística manhattan, aunque la linear nos ofrece también un tiempo muy bajo. La búsqueda en anchura y coste uniforme nos dan resultados muy similares, y en primero el mejor sucede como en A*, que manhattan nos da mejor resultado. Por último búsqueda en profundidad es el peor con mucha diferencia, probablemente porque haya empezado a buscar por ramas que no llevan a una solución.


Finalmente, en los casos E1 y E3, cabe destacar los malos resultados que da primero el mejor con ambas heurísticas, que es incluso peor que algunos de los métodos de búsqueda ciega. A* nos da de nuevo los mejores resultados (sobre todo manhattan). También cabe destacar cómo la búsqueda en profundidad en el caso E1 nos da una longitud de solución mucho mayor que en el E3, en el que el resto de métodos dan una longitud algo mayor que en el E1. Esto demuestra la importancia de la rama en la que se comience a explorar en la búsqueda en profundidad, ya que si no tiene solución hará que visitemos una gran cantidad de nodos que no nos sirven.

In [44]:
e1 = (2,1,3,4,8,6,7,0,5)
e2 = (1,0,3,4,8,6,7,2,5)
e3 = (4,5,6,1,0,3,7,8,2)
e4 = (1,2,3,0,5,6,4,7,8)

In [45]:
import time

In [46]:
start_time = time.time()
resuelve_ocho_puzzle(e1, breadth_first_graph_search)
print("--- %s seconds ---" % (time.time() - start_time))

Solución: ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 17. Nodos analizados: 14017
--- 14.557585954666138 seconds ---


In [47]:
start_time = time.time()
resuelve_ocho_puzzle(e2, breadth_first_graph_search)
print("--- %s seconds ---" % (time.time() - start_time))

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 11. Nodos analizados: 990
--- 0.06363725662231445 seconds ---


In [48]:
start_time = time.time()
resuelve_ocho_puzzle(e3, breadth_first_graph_search)
print("--- %s seconds ---" % (time.time() - start_time))

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 20. Nodos analizados: 52804
--- 221.96417021751404 seconds ---


In [49]:
start_time = time.time()
resuelve_ocho_puzzle(e4, breadth_first_graph_search)
print("--- %s seconds ---" % (time.time() - start_time))

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 3. Nodos analizados: 19
--- 0.0 seconds ---


In [50]:
start_time = time.time()
resuelve_ocho_puzzle(e1, depth_first_graph_search)
print("--- %s seconds ---" % (time.time() - start_time))

KeyboardInterrupt: 

In [ ]:
start_time = time.time()
resuelve_ocho_puzzle(e2, depth_first_graph_search)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
resuelve_ocho_puzzle(e3, depth_first_graph_search)
print("--- %s seconds ---" % (time.time() - start_time))

In [60]:
start_time = time.time()
resuelve_ocho_puzzle(e4, depth_first_graph_search)
print("--- %s seconds ---" % (time.time() - start_time))

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: depth_first_graph_search
Longitud de la solución: 3. Nodos analizados: 4
--- 0.0 seconds ---


In [ ]:
start_time = time.time()
resuelve_ocho_puzzle(e1, uniform_cost_search)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
resuelve_ocho_puzzle(e2, uniform_cost_search)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
resuelve_ocho_puzzle(e3, uniform_cost_search)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
resuelve_ocho_puzzle(e4, uniform_cost_search)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
resuelve_ocho_puzzle(e1, recursive_best_first_search, linear)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
resuelve_ocho_puzzle(e2, recursive_best_first_search, linear)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
resuelve_ocho_puzzle(e3, recursive_best_first_search, linear)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
resuelve_ocho_puzzle(e4, recursive_best_first_search, linear)
print("--- %s seconds ---" % (time.time() - start_time))

In [51]:
start_time = time.time()
resuelve_ocho_puzzle(e1, recursive_best_first_search, manhattan)
print("--- %s seconds ---" % (time.time() - start_time))

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: recursive_best_first_search
Heurística: manhattan
Longitud de la solución: 17. Nodos analizados: 573520
--- 23.41811203956604 seconds ---


In [52]:
start_time = time.time()
resuelve_ocho_puzzle(e2, recursive_best_first_search, manhattan)
print("--- %s seconds ---" % (time.time() - start_time))

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: recursive_best_first_search
Heurística: manhattan
Longitud de la solución: 11. Nodos analizados: 1261
--- 0.03289914131164551 seconds ---


In [53]:
start_time = time.time()
resuelve_ocho_puzzle(e3, recursive_best_first_search, manhattan)
print("--- %s seconds ---" % (time.time() - start_time))

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: recursive_best_first_search
Heurística: manhattan
Longitud de la solución: 22. Nodos analizados: 2303999
--- 54.43357610702515 seconds ---


In [54]:
start_time = time.time()
resuelve_ocho_puzzle(e4, recursive_best_first_search, manhattan)
print("--- %s seconds ---" % (time.time() - start_time))

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: recursive_best_first_search
Heurística: manhattan
Longitud de la solución: 3. Nodos analizados: 4
--- 0.000997781753540039 seconds ---


In [ ]:
start_time = time.time()
resuelve_ocho_puzzle(e1, astar_search, linear)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
resuelve_ocho_puzzle(e2, astar_search, linear)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
resuelve_ocho_puzzle(e3, astar_search, linear)
print("--- %s seconds ---" % (time.time() - start_time))

In [59]:
start_time = time.time()
resuelve_ocho_puzzle(e4, astar_search, linear)
print("--- %s seconds ---" % (time.time() - start_time))

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 3. Nodos analizados: 4
--- 0.000997781753540039 seconds ---


In [55]:
start_time = time.time()
resuelve_ocho_puzzle(e1, astar_search, manhattan)
print("--- %s seconds ---" % (time.time() - start_time))

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 17. Nodos analizados: 362
--- 0.04286479949951172 seconds ---


In [56]:
start_time = time.time()
resuelve_ocho_puzzle(e2, astar_search, manhattan)
print("--- %s seconds ---" % (time.time() - start_time))

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 11. Nodos analizados: 39
--- 0.0010259151458740234 seconds ---


In [57]:
start_time = time.time()
resuelve_ocho_puzzle(e3, astar_search, manhattan)
print("--- %s seconds ---" % (time.time() - start_time))

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 20. Nodos analizados: 1153
--- 0.38414859771728516 seconds ---


In [58]:
start_time = time.time()
resuelve_ocho_puzzle(e4, astar_search, manhattan)
print("--- %s seconds ---" % (time.time() - start_time))

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 3. Nodos analizados: 4
--- 0.0009975433349609375 seconds ---



### Ejercicio 6:  En el ejercicio anterior hemos hecho pruebas con distintos estados iniciales. Explica cómo afecta a la resolución del problema si lo que cambiamos es el estado objetivo.
####  En vez de  goal=(1, 2, 3, 4, 5, 6, 7, 8, 0) queremos usar goal2=(1, 2, 3, 4, 0, 5, 6, 7, 8)
####  No es necesario repetir los experimentos pero sí justificar convenientemente la respuesta dada.


La comparación entre la eficacia de las heurísticas en la resolución de los problemas seguirá siendo la misma, pero el tiempo total de resolución puede cambiar, ya que la distancia entre los problemas y este nuevo objetivo va a cambiar.


###  -----------------------------------------------------------------------------------------------------------------


### Ejercicio 7: (opcional) Definir nuevas heurísticas (más informadas) y completa una nueva fila de la tabla anterior para ver cómo afecta al número de nodos generados por los algoritmos

In [62]:
# Proponemmos como ejemplo la heurística h3 que se obtiene sumando a la heurística
# manhattan una componente que cuantifica la "secuencialidad" en las
# casillas de un tablero, al recorrerlo en el sentido de las aguas del reloj
# ¿Es h3 admisble? Comprobar cómo se comporta esta heurística cuando se usa en
# A*, con cada uno de los estados anteriores. Comentar los resultados.

# Puedes proponer otra heuristica siempre que sea más informada que todas las heurísticas ya utilizadas.

# Indices en orden del anillo exterior en la dirección de las agujas del reloj
circle_indexes = [0, 1, 2, 5, 8, 7, 6, 3]
goal_circle = [goal[i] for i in circle_indexes]
# Mapa de la posición correcta de los elementos en el anillo exterior
goal_inverse_mapping = {} 
for i in range(len(circle_indexes)):
    goal_inverse_mapping[goal_circle[i]] = i
        
        
def h3(node, reward=1, penalty=0):
    score1 = manhattan(node)
    state = node.state
    secuencialidad = 0
    current_circle = [state[i] for i in circle_indexes]
    for i in range(len(circle_indexes)):
        for j in range(i+1, len(circle_indexes)):
            prev = current_circle[i]
            next = current_circle[j]
            
            if prev == 0 or next == 0 or prev not in goal_inverse_mapping or next not in goal_inverse_mapping:
                continue

            should_prev = goal_inverse_mapping[prev]
            should_next = goal_inverse_mapping[next]

            conflict = should_prev > should_next
            secuencialidad += penalty if conflict else -reward

    return score1 + secuencialidad

def h4(node):
    return h3(node, 0, 1)

def h5(node):
    return h3(node, 0, 2)
# ---------------------------------------------------------------------------
# Que el orden de elementos en la circunferencia exterior sea el correcto, hace
# que el coste de colocarlos bien decrezca, ya que solo hay que rotar el puzzle.
# Si hay conflicto, sin embargo, el coste incrementa, ya que hay que cambiarlos
# de orden. Por ello hemos probado dos posibles implementaciones, penalizando 
# conflictos o premiando la falta de ellos.
# Como el coste de cambiar dos elementos de orden tiene coste 2, hemos probado
# dos variaciones de posibles penalizaciones.

In [72]:
p1,p2,p3 = (2, 4, 3, 1, 5, 6, 7, 8, 0), (1, 2, 3, 4, 5, 6, 0, 7, 8), (1, 2, 3, 4, 5, 7, 8, 6, 0)
es = [e1,e2,e3,e4,p1,p2,p3]
hs = [manhattan, h3, h4, h5]
for e in es:
    print("\nProblema: {}".format(e))
    for h in hs:
        start_time = time.time()
        print(resuelve_ocho_puzzle(e, astar_search, h))
        print("--- %s seconds ---" % (time.time() - start_time))


Problema: (2, 1, 3, 4, 8, 6, 7, 0, 5)
Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 17. Nodos analizados: 362
None
--- 0.04933977127075195 seconds ---
Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heu

In [1]:
%%timeit
for e in es:
    astar_search(Ocho_Puzzle(e), manhattan)

NameError: name 'es' is not defined

In [69]:
%%timeit
for e in es:
    astar_search(Ocho_Puzzle(e), h3)

39.4 ms ± 8.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [70]:
%%timeit
for e in es:
    astar_search(Ocho_Puzzle(e), h4)

50.5 ms ± 6.63 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [71]:
%%timeit
for e in es:
    astar_search(Ocho_Puzzle(e), h5)

84.2 ms ± 14.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Observando los resultados, en general siempre parece mejor h3 que manhattan, pero dependiendo del caso, h4 y h5 pueden ser mejores o peores que manhattan. Probablemente porque haya casos donde movimientos son evaluados peores de lo que son por culpa de las penalizaciones, aunque en algunos casos disminuyen por mucho la cantidad de estados visitados.

### Ejercicio 8. Puzle de 15.
En 1878, Sam Loyd daba un premio de 1.000 dolares a quien fuera capaz de resolver su famoso 15 Puzzle (n=16). Se trataba de un puzzle deslizante de 16 piezas que *no tenía solución* debido que se necesitaba un número impar de movimientos y, como hemos visto, sólo un número de movimientos par tiene solución. Muchas personas trataron de resolverlo. 
El número de posibles estados iniciales es n!, siendo n el número de fichas (números y hueco). Por tanto, en el puzle 4x4 (16 fichas), tendremos más de 130.000 millones de posibles estados iniciales. Sin embargo, sólo la mitad de esas combinaciones tiene solución. 
En el caso del siguiente estado inicial que sí tiene solución:  (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0) indica los nodos generados por A* con alguna de las heurísticas y comparalo con el puzle de 8.


Emplearemos la heurísitca manhattan al ser más realista que la linear.
Como vemos, en el nodo inicial tiene 2 nodos hijos:

    (1,2,3,4,5,6,7,8,9,10,12,15,13,14,0,11) con valor heurístico 1+2+2+1=6
    
    (1,2,3,4,5,6,7,8,9,10,12,0,13,14,11,15) con valor heurísitico 1+2+1+1=5

Por tanto explorará los hijos del segundo nodo, que son:

    (1,2,3,4,5,6,7,8,9,10,0,12,13,14,11,15) valor heurístico 4
    
    (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0) que es el mismo que el estado inicial
    
    (1,2,3,4,5,6,7,0,9,10,12,8,13,14,11,15) valor heurístico 6

Seguimos explorando el primer hijo, cuyos nodos hijos son:

    (1,2,3,4,5,6,7,8,9,10,11,12,13,14,0,15) valor 2
    
    (1,2,3,4,5,6,7,8,9,10,12,0,13,14,11,15) valor 4 
    
    (1,2,3,4,5,6,0,8,9,10,7,12,13,14,11,15) valor 6
    
    (1,2,3,4,5,6,7,8,9,0,10,12,13,14,11,15) valor 5

Seguimos con el primero:

    (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0) que es el objetivo
    